In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime,timedelta

from sklearn.metrics import  mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score

def mbe(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    y_true = y_true.reshape( len(y_true),1 )
    y_pred = y_pred.reshape( len(y_pred),1 )
   
    diff = ( y_true - y_pred )
    mbe = diff.mean()
    return mbe

In [13]:
%pwd

'D:\\Data\\solar\\code'

In [16]:
#read solar_data
data = pd.read_csv('../data/processed/2019_bon_solar_data.csv')


In [42]:
SAVE_PATH = "../result/m1/0905/gs/"

if os.path.exists(SAVE_PATH) == 0:
    os.makedirs(SAVE_PATH)
    
LAGGED_HOUR = False
PREMODEL = True
maxVal = 1.5 # kt_maxVal


In [43]:
data_ = data[1:].copy()
print('data_.shape:',data_.shape)
data_.head()

data_.shape: (524918, 10)


,Time,zen,dw_solar,direct_n,diffuse,dw_ir,temp,rh,windspd,pressure
1,2019-01-01 00:01:00,105.28,-2.1,0.4,-0.1,336.2,5.5,93.1,10.7,978.8
2,2019-01-01 00:02:00,105.46,-2.1,0.4,-0.1,336.3,5.5,93.1,10.5,978.8
3,2019-01-01 00:03:00,105.65,-2.1,0.4,0.0,335.8,5.5,93.2,10.2,978.8
4,2019-01-01 00:04:00,105.83,-2.1,0.4,0.0,335.3,5.4,93.2,10.4,978.8
5,2019-01-01 00:05:00,106.01,-2.1,0.4,0.0,335.1,5.4,93.0,10.5,978.8


In [22]:
def Mean_Matrix(data,step):
    
    a = np.arange(len(data)).tolist()
    step = step
    b = [a[i:i+step] for i in range(0,len(a),step)] #分组

    dfs = list()
    for l in tqdm(b):
        dfs.append(data.iloc[l].mean(axis=0))
    mean_ = pd.concat(dfs,  axis=1).T
    mean_.columns =data.columns
    return mean_

In [23]:
time = datetime(2019,1,1,0,1,)
time_= datetime(2020,1,1,0,0,)
date_range = pd.date_range(start=time, end=time_, freq="1min")  # freq="D"表示按天，可以按分钟，月，季度，年等

In [24]:
data_.loc[:,"Time"] = data_.loc[:,"Time"].astype('datetime64[s]')
data_ = data_.set_index("Time")

data_ = data_.reindex(index=date_range) 
print(data_.shape)

(525600, 9)


In [25]:
#
data_[data_<-1000] = np.NaN
data_[data_.loc[:,['dw_solar', 'direct_n', 'diffuse', 'dw_ir']]<0] = 0

In [26]:
mean_data = Mean_Matrix(data_,10)

100%|██████████████████████████████████████████████████████████████████████████| 52560/52560 [00:31<00:00, 1664.72it/s]


In [27]:
time = datetime(2019,1,1,0,10,)
time_= datetime(2020,1,1,0,0,)
date_range_hour = pd.date_range(start=time, end=time_, freq="10min")  # freq="D"表示按天，可以按分钟，月，季度，年等

In [28]:
mean_data.index = date_range_hour

times = mean_data.index
times = pd.DatetimeIndex(times)

In [31]:
#clear_sky
cs = pd.read_csv("../data/bon_clear_sky_2019.csv",index_col = 0)

In [32]:
cs=cs[1:]
print(cs.shape)

mean_cs = Mean_Matrix(cs,10)
print(mean_cs.shape)
mean_cs.head()

(525599, 4)


100%|██████████████████████████████████████████████████████████████████████████| 52560/52560 [00:27<00:00, 1919.67it/s]


(52560, 4)


,DNI_McClear,GHI_McClear,DNI_REST2,GHI_REST2
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [33]:
ghi_clc=mean_cs['GHI_McClear']
dni_clc=mean_cs['DNI_McClear']

In [34]:
mean_data['ghi_clc'] = ghi_clc.to_numpy()
mean_data['dni_clc'] = dni_clc.to_numpy()

mean_data['kt_ghi'] = mean_data['dw_solar']/(mean_data['ghi_clc']+0.01)
mean_data['kt_dni'] = mean_data['direct_n']/(mean_data['dni_clc']+0.01)

In [35]:
mean_data.loc[mean_data['kt_ghi']>= maxVal,'kt_ghi']= maxVal
mean_data.loc[mean_data['kt_dni']>= maxVal,'kt_dni']= maxVal

In [36]:
#ghi & dni
kt_ghi_target_cols = ['kt_ghi_target10','kt_ghi_target20','kt_ghi_target30','kt_ghi_target40','kt_ghi_target50','kt_ghi_target60','kt_ghi_target70', 'kt_ghi_target80','kt_ghi_target90','kt_ghi_target100','kt_ghi_target110','kt_ghi_target120','kt_ghi_target130','kt_ghi_target140','kt_ghi_target150','kt_ghi_target160','kt_ghi_target170','kt_ghi_target180']
kt_dni_target_cols = ['kt_dni_target10','kt_dni_target20','kt_dni_target30','kt_dni_target40','kt_dni_target50','kt_dni_target60','kt_dni_target70', 'kt_dni_target80','kt_dni_target90','kt_dni_target100','kt_dni_target110','kt_dni_target120','kt_dni_target130','kt_dni_target140','kt_dni_target150','kt_dni_target160','kt_dni_target170','kt_dni_target180']
ghi_lag_cols=['ghi_lag0','ghi_lag10','ghi_lag20','ghi_lag30','ghi_lag40','ghi_lag50','ghi_lag60','ghi_lag70','ghi_lag80','ghi_lag90','ghi_lag100','ghi_lag110','ghi_lag120','ghi_lag130','ghi_lag140','ghi_lag150','ghi_lag160','ghi_lag170','ghi_lag180']
dni_lag_cols=['dni_lag0','dni_lag10','dni_lag20','dni_lag30','dni_lag40','dni_lag50','dni_lag60','dni_lag70','dni_lag80','dni_lag90','dni_lag100','dni_lag110','dni_lag120','dni_lag130','dni_lag140','dni_lag150','dni_lag160','dni_lag170','dni_lag180']

In [37]:
kt_ghi = pd.DataFrame(np.full((mean_data.shape[0],len(kt_ghi_target_cols)), -9999), columns=kt_ghi_target_cols)
kt_dni = pd.DataFrame(np.full((mean_data.shape[0],len(kt_dni_target_cols)), -9999),columns=kt_dni_target_cols)
ghi_lag = pd.DataFrame(np.full((mean_data.shape[0],len(ghi_lag_cols)), -9999),columns=ghi_lag_cols)
dni_lag = pd.DataFrame(np.full((mean_data.shape[0],len(dni_lag_cols)), -9999),columns=dni_lag_cols)

In [38]:
mean_data_ = mean_data.copy()
mean_data_.index = range(len(mean_data_))

In [39]:
for i in range(len(kt_ghi_target_cols)):
        kt_ghi.iloc[len(kt_ghi_target_cols)-1-i:-1-i,i] = mean_data_.loc[len(kt_ghi_target_cols):,'kt_ghi']
for i in range(len(kt_dni_target_cols)):
        kt_dni.iloc[len(kt_dni_target_cols)-1-i:-1-i,i] = mean_data_.loc[len(kt_dni_target_cols):,'kt_dni']

#for i in range(len(ghi_lag_list)):
#    if i == 0:
#        ghi_lag.loc[len(ghi_lag_list)-1+i:,ghi_lag_list[i]] = data_['kt_ghi'][len(ghi_lag_list)-1:]
#    else:
#        ghi_lag.loc[len(ghi_lag_list)-1+i:,ghi_lag_list[i]] = data_['kt_ghi'][len(ghi_lag_list)-1:-i]
        
for i in range(len(ghi_lag_cols)):
    if i == 0:
        ghi_lag.loc[:,ghi_lag_cols[i]][len(ghi_lag_cols)-1+i:] = mean_data_['kt_ghi'][len(ghi_lag_cols)-1:]
    else:
        ghi_lag.loc[:,ghi_lag_cols[i]][len(ghi_lag_cols)-1+i:] = mean_data_['kt_ghi'][len(ghi_lag_cols)-1:-i]        
        

#for i in range(len(dni_lag_list)):
#    if i == 0:
#        dni_lag.loc[len(dni_lag_list)-1+i:,dni_lag_list[i]] = data_['kt_dni'][len(dni_lag_list)-1:]
#    else:
#        dni_lag.loc[len(dni_lag_list)-1+i:,dni_lag_list[i]] = data_['kt_dni'][len(dni_lag_list)-1:-i]
for i in range(len(dni_lag_cols)):
    if i == 0:
        dni_lag.loc[:,dni_lag_cols[i]][len(dni_lag_cols)-1+i:] = mean_data_['kt_dni'][len(dni_lag_cols)-1:]
    else:
        dni_lag.loc[:,dni_lag_cols[i]][len(dni_lag_cols)-1+i:] = mean_data_['kt_dni'][len(dni_lag_cols)-1:-i]

<ipython-input-39-eaaa2e9e4330>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ghi_lag.loc[:,ghi_lag_cols[i]][len(ghi_lag_cols)-1+i:] = mean_data_['kt_ghi'][len(ghi_lag_cols)-1:-i]
<ipython-input-39-eaaa2e9e4330>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dni_lag.loc[:,dni_lag_cols[i]][len(dni_lag_cols)-1+i:] = mean_data_['kt_dni'][len(dni_lag_cols)-1:-i]


In [45]:
if LAGGED_HOUR == True:
    ghi_lag_ = pd.DataFrame(np.full((mean_data_hour.shape[0],len(ghi_lag_cols)), -9999),columns=ghi_lag_cols)
    dni_lag_ = pd.DataFrame(np.full((mean_data_hour.shape[0],len(dni_lag_cols)), -9999),columns=dni_lag_cols)

    first_hour_ghi = ["ghi_lag10","ghi_lag20","ghi_lag30","ghi_lag40","ghi_lag50","ghi_lag60"]
    second_hour_ghi =["ghi_lag70","ghi_lag80","ghi_lag90","ghi_lag100","ghi_lag110","ghi_lag120"]
    third_hour_ghi = ["ghi_lag130","ghi_lag140","ghi_lag150","ghi_lag160","ghi_lag170","ghi_lag180"]

    first_hour_dni = ["dni_lag10","dni_lag20","dni_lag30","dni_lag40","dni_lag50","dni_lag60"]
    second_hour_dni =["dni_lag70","dni_lag80","dni_lag90","dni_lag100","dni_lag110","dni_lag120"]
    third_hour_dni=["dni_lag130","dni_lag140","dni_lag150","dni_lag160","dni_lag170","dni_lag180"]

    for col in first_hour_ghi:
        ghi_lag_.loc[:,col] = ghi_lag.loc[:,'ghi_lag60']
    for col in second_hour_ghi:
        ghi_lag_.loc[:,col] = ghi_lag.loc[:,'ghi_lag120']
    for col in third_hour_ghi:
        ghi_lag_.loc[:,col] = ghi_lag.loc[:,'ghi_lag180']
    
    ghi_lag_.loc[:,'ghi_lag0'] = ghi_lag.loc[:,'ghi_lag0']

    for col in first_hour_dni:
        dni_lag_.loc[:,col] = dni_lag.loc[:,'dni_lag60']
    for col in second_hour_dni:
        dni_lag_.loc[:,col] = dni_lag.loc[:,'dni_lag120']
    for col in third_hour_dni:
        dni_lag_.loc[:,col] = dni_lag.loc[:,'dni_lag180']
    
    dni_lag_.loc[:,'dni_lag0'] = dni_lag.loc[:,'dni_lag0']

    data_2 = pd.concat([mean_data_,kt_ghi,kt_dni,ghi_lag_,dni_lag_],axis = 1)
    print(data_2.shape)
else:
    data_2 = pd.concat([mean_data_,kt_ghi,kt_dni,ghi_lag,dni_lag],axis = 1)
    print(data_2.shape)

(52560, 87)


In [46]:
data_2['No'] = [i for i in range(len(data_2))]
data_2.index = mean_data.index

data_2.replace(-9999,np.NaN,inplace = True)
data_2.dropna(axis=0,inplace = True)

print(data_2.shape)

(51183, 88)


In [48]:
#drop zenith > 85
drop_zen_time = data_2[data_2.loc[:,'zen']>85].index.to_list()
len(drop_zen_time)

27625

In [49]:
#检索/保存目标zen_time在3小时内的索引
#时间太长，可优化
drop_zen_time_3h = list()
for i in tqdm(range(len(drop_zen_time))):
    start = drop_zen_time[i] +timedelta(hours=-3)
    end= drop_zen_time[i] + timedelta(hours=3)
    if (drop_zen_time[i] not in drop_zen_time_3h) or (start not in drop_zen_time_3h) or (end not in drop_zen_time_3h):
        range_hour = pd.date_range(start=start, end=end, freq="10min") #.to_list()
        for t in range_hour:
            if t not in drop_zen_time_3h:
                drop_zen_time_3h.append(t)

100%|████████████████████████████████████████████████████████████████████████████| 27625/27625 [26:18<00:00, 17.50it/s]


In [50]:
#drop drop_zen_time_3h
for i in tqdm(pd.DatetimeIndex(drop_zen_time_3h)):
    if  i in data_2.index:
      data_2.drop(i,axis=0,inplace = True)

data_3 = data_2.copy()
print(data_3.shape)

#检查是否存在空值
data_3.isna().any().sum()
print(data_3.shape)

100%|████████████████████████████████████████████████████████████████████████████| 40852/40852 [08:04<00:00, 84.34it/s]

(10910, 88)
(10910, 88)


In [51]:
data_4 = data_3.copy()
data_4['time']  = data_4.index

In [52]:
gm1 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==1:
        gm1.append(i)
gm2 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==2:
        gm2.append(i)
gm3 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==3:
        gm3.append(i)
gm4 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==4:
        gm4.append(i)
gm5 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==5:
        gm5.append(i)
gm6 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==6:
        gm6.append(i)
gm7 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==7:
        gm7.append(i)
gm8 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==8:
        gm8.append(i)
gm9 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==9:
        gm9.append(i)
gm10 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==10:
        gm10.append(i)
gm11 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==11:
        gm11.append(i)
gm12 = []
for i in range(len(data_4[:])):
    if data_4.iloc[i,-1].month ==12:
        gm12.append(i)

In [53]:
train_index = gm1[:int(len(gm1)*0.7)]+gm2[:int(len(gm2)*0.7)]+gm3[:int(len(gm3)*0.7)]+gm4[:int(len(gm4)*0.7)]+gm5[:int(len(gm5)*0.7)]+gm6[:int(len(gm6)*0.7)]+gm7[:int(len(gm7)*0.7)]+gm8[:int(len(gm8)*0.7)]+gm9[:int(len(gm9)*0.7)]+gm10[:int(len(gm10)*0.7)]+gm11[:int(len(gm11)*0.7)]+gm12[:int(len(gm12)*0.7)]
test_index = gm1[int(len(gm1)*0.7):]+gm2[int(len(gm2)*0.7):]+gm3[int(len(gm3)*0.7):]+gm4[int(len(gm4)*0.7):]+gm5[int(len(gm5)*0.7):]+gm6[int(len(gm6)*0.7):]+gm7[int(len(gm7)*0.7):]+gm8[int(len(gm8)*0.7):]+gm9[int(len(gm9)*0.7):]+gm10[int(len(gm10)*0.7):]+gm11[int(len(gm11)*0.7):]+gm12[int(len(gm12)*0.7):]

In [54]:


#save_InputandOutput
data_4.to_csv(SAVE_PATH+"bon_10m.csv",index=0)
data_4.iloc[train_index].to_csv(SAVE_PATH+"bon_train_10m.csv",index=0)
data_4.iloc[test_index].to_csv(SAVE_PATH+"bon_test_10m.csv",index=0)

In [55]:
#clc_target

In [56]:
ghi_clc_cols = ['ghi_clc1','ghi_clc2','ghi_clc3','ghi_clc4','ghi_clc5','ghi_clc6','ghi_clc7','ghi_clc8','ghi_clc9','ghi_clc10','ghi_clc11','ghi_clc12','ghi_clc13','ghi_clc14','ghi_clc15','ghi_clc16','ghi_clc17','ghi_clc18']
dni_clc_cols = ['dni_clc1','dni_clc2','dni_clc3','dni_clc4','dni_clc5','dni_clc6','dni_clc7','dni_clc8','dni_clc9','dni_clc10','dni_clc11','dni_clc12','dni_clc13','dni_clc14','dni_clc15','dni_clc16','dni_clc17','dni_clc18']

In [57]:
test_set = data_4.iloc[test_index]
train_set = data_4.iloc[train_index]

In [58]:
mean_data_['No'] = [i for i in range(len(mean_data_))]

In [59]:
ghi_clc_list = []
dni_clc_list = []
for num in range(len(kt_ghi_target_cols)):
    t = num + 1
    target_index = test_set.loc[:,'No'] + t 
    ghi_data = mean_data_.loc[target_index,'ghi_clc'].to_numpy()
    dni_data = mean_data_.loc[target_index,'dni_clc'].to_numpy()
    ghi_clc_list.append(ghi_data)
    dni_clc_list.append(dni_data)


In [60]:
K =ghi_clc_list
K = list(map(list, zip(*K)))#转置

L = dni_clc_list
L = list(map(list, zip(*L)))#转置

In [61]:
data_ghi = pd.DataFrame(np.array(K),columns = ghi_clc_cols,index = test_set["time"])
data_dni = pd.DataFrame(np.array(L),columns = dni_clc_cols,index = test_set["time"])
data_target_clc = pd.concat([data_ghi,data_dni],axis =1)

In [62]:
data_target_clc.to_csv(SAVE_PATH+'target_clc_10m.csv')

In [64]:
s_data2 = pd.read_csv('../data/BON.csv')
s_data2.rename(columns={'Unnamed: 0': 'time'},inplace =True)
datetimes = s_data2["time"]
datetimes =pd.DatetimeIndex(datetimes)

In [65]:
#s_data2.to_csv("concatdata\\bon\\BON.csv",index = 0)
s_data2["time"] = pd.DatetimeIndex(datetimes)
s_data2.set_index('time')
s_data2.index = s_data2["time"]
s_data2.drop('time',axis = 1,inplace = True)

In [66]:
def ch_cols(Data,target_cols,name):
    target_ =list()
    for i in range(len(Data.columns)):
        col_name = Data.columns[i]
        col_split = col_name.split('_')[0]
        if col_split == name:
            target_.append(col_name)
    return target_

In [67]:
ch_01 = ch_cols(s_data2,s_data2.columns,'C01')
ch_02 = ch_cols(s_data2,s_data2.columns,'C02')
ch_03 = ch_cols(s_data2,s_data2.columns,'C03')
ch_04 = ch_cols(s_data2,s_data2.columns,'C04')
ch_05 = ch_cols(s_data2,s_data2.columns,'C05')

In [68]:
from sklearn.preprocessing import MinMaxScaler
def MinMax_Ch(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler.transform(data)
    

In [69]:
target_ch_cols = ch_01+ch_02+ch_03+ch_04+ch_05
s_data3 = s_data2.loc[:,target_ch_cols]

In [70]:
s_data3.loc[:,ch_01]= MinMax_Ch(s_data3.loc[:,ch_01])
s_data3.loc[:,ch_02]= MinMax_Ch(s_data3.loc[:,ch_02])
s_data3.loc[:,ch_03]= MinMax_Ch(s_data3.loc[:,ch_03])
s_data3.loc[:,ch_04]= MinMax_Ch(s_data3.loc[:,ch_04])
s_data3.loc[:,ch_05]= MinMax_Ch(s_data3.loc[:,ch_05])

In [81]:
#train_set = pd.read_csv("/content/drive/MyDrive/solar/test/0901_ini/gs/bon_train_10m_MC.csv",index_col = 'time')
#test_set = pd.read_csv('/content/drive/MyDrive/solar/test/0901_ini/gs/bon_test_10m_MC.csv',index_col = 'time')
#data_target_clc =  pd.read_csv("/content/drive/MyDrive/solar/test/0901_ini/gs/target_clc_10m_MC.csv",index_col = 'time')
persistence = pd.read_csv('../data/persistence_MC.csv')

train_set.index = pd.DatetimeIndex(train_set.index)
test_set.index = pd.DatetimeIndex(test_set.index)

In [72]:
data_target_clc.index = pd.DatetimeIndex(data_target_clc.index )
s_data3.iloc[train_set.No].index[0] == train_set.index[0]

True

In [73]:
train_set = pd.concat([train_set,s_data3.iloc[train_set.No]],axis =1 )
test_set = pd.concat([test_set,s_data3.iloc[test_set.No]],axis =1 )

In [74]:
kt_ghi_target_cols = ['kt_ghi_target10','kt_ghi_target20','kt_ghi_target30','kt_ghi_target40','kt_ghi_target50','kt_ghi_target60','kt_ghi_target70', 'kt_ghi_target80','kt_ghi_target90','kt_ghi_target100','kt_ghi_target110','kt_ghi_target120','kt_ghi_target130','kt_ghi_target140','kt_ghi_target150','kt_ghi_target160','kt_ghi_target170','kt_ghi_target180']
kt_dni_target_cols = ['kt_dni_target10','kt_dni_target20','kt_dni_target30','kt_dni_target40','kt_dni_target50','kt_dni_target60','kt_dni_target70', 'kt_dni_target80','kt_dni_target90','kt_dni_target100','kt_dni_target110','kt_dni_target120','kt_dni_target130','kt_dni_target140','kt_dni_target150','kt_dni_target160','kt_dni_target170','kt_dni_target180']
ghi_lag_cols=['ghi_lag0','ghi_lag10','ghi_lag20','ghi_lag30','ghi_lag40','ghi_lag50','ghi_lag60','ghi_lag70','ghi_lag80','ghi_lag90','ghi_lag100','ghi_lag110','ghi_lag120','ghi_lag130','ghi_lag140','ghi_lag150','ghi_lag160','ghi_lag170','ghi_lag180']
dni_lag_cols=['dni_lag0','dni_lag10','dni_lag20','dni_lag30','dni_lag40','dni_lag50','dni_lag60','dni_lag70','dni_lag80','dni_lag90','dni_lag100','dni_lag110','dni_lag120','dni_lag130','dni_lag140','dni_lag150','dni_lag160','dni_lag170','dni_lag180']

In [75]:
print("train_set",train_set.shape)
print("test_set",test_set.shape)
test_set['No'] = range(len(test_set))
train_set_drop = train_set.dropna(axis=0)
test_set_drop = test_set.dropna(axis=0)

print("train_set",train_set_drop.shape)
print("test_set",test_set_drop.shape)

train_set (7631, 694)
test_set (3279, 694)
train_set (5746, 694)
test_set (2296, 694)


In [76]:
target_cols = kt_ghi_target_cols + kt_dni_target_cols
train_cols = ghi_lag_cols +dni_lag_cols +target_ch_cols

train_X = train_set_drop[train_cols]
train_Y = train_set_drop[target_cols]

test_X = test_set_drop[train_cols]
test_Y = test_set_drop[target_cols]


print("train_X",train_X.shape)
print("train_Y",train_Y.shape)

print("test_X",test_X.shape)
print("test_Y",test_Y.shape)


train_X (5746, 643)
train_Y (5746, 36)
test_X (2296, 643)
test_Y (2296, 36)


In [77]:
target_clc = data_target_clc.iloc[test_set_drop.No]

In [78]:
train_set_drop[target_cols+train_cols].to_csv(SAVE_PATH+'train_set_drop.csv')
test_set_drop[target_cols+train_cols].to_csv(SAVE_PATH+'test_set_drop.csv')

In [82]:
per_rmse = persistence['rmse']

In [83]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.preprocessing import MinMaxScaler
#from keras.wrappers.scikit_learn import KerasRegressor
#import scikeras
#from scikeras.wrappers import KerasRegressor
#from sklearn.model_selection import GridSearchCV

In [85]:
target_cols = kt_ghi_target_cols + kt_dni_target_cols
lagged_cols = ghi_lag_cols +dni_lag_cols + target_ch_cols

df_for_training = train_set_drop[target_cols+lagged_cols]
df_for_testing = test_set_drop[target_cols+lagged_cols]

print("df_for_training",df_for_training.shape)
print("df_for_testing",df_for_testing.shape)



df_for_training (5746, 679)
df_for_testing (2296, 679)


In [86]:
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled = scaler.transform(df_for_testing)

In [89]:
Just_ghi = False
if Just_ghi:
    target_num  = int(len(target_cols)/2)
else:
    target_num  = len(target_cols)

In [90]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past,len(dataset)):
            dataX.append(dataset[i - n_past:i, len(target_cols):dataset.shape[1]])
            dataY.append(dataset[i,:target_num])
    return np.array(dataX),np.array(dataY)
trainX,trainY=createXY(df_for_training_scaled,1)
testX,testY=createXY(df_for_testing_scaled,1)

In [91]:
print("trainX",trainX.shape)
print("trainY",trainY.shape)
print("testX",testX.shape)
print("testY",testY.shape)

trainX (5745, 1, 643)
trainY (5745, 36)
testX (2295, 1, 643)
testY (2295, 36)


In [92]:
target_clc = data_target_clc.iloc[test_set_drop.No].iloc[:testY.shape[0],:target_num].to_numpy()
print(target_clc.shape)

(2295, 36)


In [93]:
grid_model = Sequential()
grid_model.add(LSTM(200,return_sequences=True,input_shape=(trainX.shape[1],trainX.shape[2])))
grid_model.add(LSTM(50))
#grid_model.add(Dropout(0.2))
grid_model.add(Dense(target_num))
    #adam=Adam(learning_rate=0.03)
grid_model.compile(loss = 'mse',optimizer = 'adam')
grid_model.fit(trainX,trainY,batch_size =1000,epochs = 200)

prediction=grid_model.predict(testX)
print("prediction\n", prediction)
print("\nPrediction Shape-",prediction.shape)
print(r2_score(testY,prediction))

Epoch 1/200
6/6 [==============================] - 5s 57ms/step - loss: 0.2183
Epoch 2/200
6/6 [==============================] - 0s 56ms/step - loss: 0.1230
Epoch 3/200
6/6 [==============================] - 0s 57ms/step - loss: 0.0820
Epoch 4/200
6/6 [==============================] - 0s 53ms/step - loss: 0.0669
Epoch 5/200
6/6 [==============================] - 0s 88ms/step - loss: 0.0581
Epoch 6/200
6/6 [==============================] - 0s 73ms/step - loss: 0.0510
Epoch 7/200
6/6 [==============================] - 0s 60ms/step - loss: 0.0458
Epoch 8/200
6/6 [==============================] - 0s 55ms/step - loss: 0.0427
Epoch 9/200
6/6 [==============================] - 0s 55ms/step - loss: 0.0406
Epoch 10/200
6/6 [==============================] - 0s 57ms/step - loss: 0.0391
Epoch 11/200
6/6 [==============================] - 0s 64ms/step - loss: 0.0381
Epoch 12/200
6/6 [==============================] - 0s 59ms/step - loss: 0.0376
Epoch 13/200
6/6 [==============================]

In [94]:
per_rmse = per_rmse[:target_num]

In [95]:
#1001
mbe_list = list() 
mae = list()
rmse = list()
r2 =  list()

y_pre_clc = prediction*target_clc
y_t_clc = testY*target_clc


for i in range(target_num):
    mae.append(MAE(y_t_clc[:,i],y_pre_clc[:,i]))
    rmse.append(np.sqrt(MSE(y_t_clc[:,i],y_pre_clc[:,i])))
    r2.append(r2_score(y_t_clc[:,i],y_pre_clc[:,i]))
    mbe_list.append(mbe(y_t_clc[:,i],y_pre_clc[:,i]))

s = 1-(np.array(rmse)/np.array(per_rmse))
data_lstm = pd.DataFrame([mbe_list,mae,rmse,r2,s],index=["mbe","mae", "rmse", "r2",'s'])
data_lstm=data_lstm.T
#s_score.append(data_lstm.loc[:,'s'])
data_lstm

,mbe,mae,rmse,r2,s
0,-11.314237,60.108260,87.007159,0.851185,0.224101
1,-15.670892,65.892022,93.855439,0.826846,0.320773
2,-12.451231,69.050943,98.280259,0.810411,0.337835
3,-15.498216,72.294150,102.131765,0.796232,0.344415
4,-10.392553,75.540467,105.588654,0.782310,0.356231
5,-10.221451,74.504368,103.328372,0.770952,0.396389
6,-14.793807,76.098141,105.084465,0.763546,0.406239
7,-12.836079,76.806483,106.352271,0.758158,0.405055
8,-13.208638,77.557208,107.546101,0.753390,0.407489
9,-10.278512,78.654222,109.023745,0.747312,0.403003


In [96]:
data_lstm.to_csv(SAVE_PATH+'lstm'+'_10m.csv',index = 0)